In [5]:
import os
import re
from pathlib import Path

def estandarizar_nombres_experimento(ruta_directorio, numero_exp, data_id=None, eliminar_fecha=True, simulacro=True):
    ruta = Path(ruta_directorio)
    
    if not ruta.exists():
        print(f"❌ Error: La ruta '{ruta}' no existe.")
        return

    print(f"--- Procesando: {ruta.name} (Simulacro: {simulacro}) ---")

    # 1. Patrón para FECHA
    regex_fecha_str = r'_\d{8}_\d{6}'
    patron_fecha = re.compile(regex_fecha_str)

    # 2. Patrón EXP + BASURA + CHUNK
    patron_con_chunk = re.compile(r'exp_\d+.*?_chunk')

    # 3. Patrón EXP + BASURA + FECHA
    patron_con_fecha = re.compile(r'exp_\d+.*?' + regex_fecha_str)

    # 4. Patrón EXP + RESTO (Corrección para tu caso)
    # Encuentra "exp_N" y todo el texto que le sigue hasta llegar al punto de la extensión.
    # [^.]* significa "cualquier cosa que no sea un punto".
    patron_exp_greedy = re.compile(r'exp_\d+[^.]*')

    # 5. Patrón simple (Solo números, por si acaso falla lo anterior)
    patron_exp_simple = re.compile(r'exp_\d+')

    archivos_modificados = 0

    for archivo in ruta.iterdir():
        if archivo.is_file():
            nombre_viejo = archivo.name
            nombre_nuevo = nombre_viejo

            # --- A. CASO: TENEMOS UN DATA_ID PARA INSERTAR ---
            if data_id:
                
                # PRIORIDAD 1: Es un archivo tipo CHUNK
                if 'chunk' in nombre_nuevo and patron_con_chunk.search(nombre_nuevo):
                    nombre_nuevo = patron_con_chunk.sub(
                        f'exp_{numero_exp}_{data_id}_chunk', 
                        nombre_nuevo
                    )

                # PRIORIDAD 2: No es chunk, pero TIENE FECHA
                elif patron_con_fecha.search(nombre_nuevo):
                    if eliminar_fecha:
                        nombre_nuevo = patron_con_fecha.sub(
                            f'exp_{numero_exp}_{data_id}', 
                            nombre_nuevo
                        )
                    else:
                        pass # Aquí podrías añadir lógica si decides no borrar fecha

                # PRIORIDAD 3: Limpieza completa (EL ARREGLO)
                # Si no tiene fecha ni es chunk, asumimos que todo lo que hay después 
                # del exp_N es el "data_id" viejo y queremos reemplazarlo por el nuevo.
                else:
                    # Usamos el patrón greedy que se "come" el sufijo viejo (ej: _simulation_testing)
                    nombre_nuevo = patron_exp_greedy.sub(
                        f'exp_{numero_exp}_{data_id}', 
                        nombre_nuevo
                    )

            # --- B. CASO: SOLO ACTUALIZAR NUMERO EXP (Sin Data_ID) ---
            else:
                 nombre_nuevo = patron_exp_simple.sub(f'exp_{numero_exp}', nombre_nuevo)

            # --- C. LIMPIEZA FINAL DE FECHA (Seguridad) ---
            if eliminar_fecha and patron_fecha.search(nombre_nuevo):
                 nombre_nuevo = patron_fecha.sub('', nombre_nuevo)

            # --- EJECUTAR ---
            if nombre_nuevo != nombre_viejo:
                if simulacro:
                    print(f"📝 [SIM] {nombre_viejo} \n       -> {nombre_nuevo}")
                else:
                    try:
                        archivo.rename(ruta / nombre_nuevo)
                        print(f"✅ [OK] {nombre_nuevo}")
                    except OSError as e:
                        print(f"❌ [Error] {e}")
                
                archivos_modificados += 1

    print(f"\nArchivos afectados: {archivos_modificados}")

In [7]:
# 1. Definir la ruta
script_path = os.getcwd()
project_path = os.path.join(script_path, '..')

DATA_ID = 'simulation_testing'
EXPERIMENT_ID = 'experiment_4b'
folder_path = os.path.join(project_path, 'results', EXPERIMENT_ID, DATA_ID)

# 2. Ejecutar primero en modo SIMULACRO (recomendado)
# Esto cambiará 'exp_X' a 'exp_2' y borrará la fecha.
estandarizar_nombres_experimento(
    ruta_directorio=folder_path, 
    numero_exp=EXPERIMENT_ID.split('_')[-1], 
    data_id=DATA_ID,
    eliminar_fecha=True, 
    simulacro=False  # <--- Cambia a False cuando estés listo
)

--- Procesando: simulation_testing (Simulacro: False) ---
✅ [OK] results_exp_4b_simulation_testing.pkl
✅ [OK] results_exp_4b_simulation_testing_chunk_0.pkl
✅ [OK] results_exp_4b_simulation_testing_chunk_1.pkl

Archivos afectados: 3
